In [2]:
import scipy.io as sio
from scipy import signal
import numpy as np
import pandas as pd
from sklearn import preprocessing as pre

In [3]:
def preprocessing(input,feature):
    overall=signal.firwin(9,[0.0625,0.46875],window='hamming')
    theta=signal.firwin(9,[0.0625,0.125],window='hamming')
    alpha=signal.firwin(9,[0.125,0.203125],window='hamming')
    beta=signal.firwin(9,[0.203125,0.46875],window='hamming')
    filtedData=signal.filtfilt(overall,1,input)
    filtedtheta=signal.filtfilt(theta,1,filtedData)
    filtedalpha=signal.filtfilt(alpha,1,filtedData)
    filtedbeta=signal.filtfilt(beta,1,filtedData)
    ftheta,psdtheta=signal.welch(filtedtheta,nperseg=256)
    falpha,psdalpha=signal.welch(filtedalpha,nperseg=256)
    fbeta,psdbeta=signal.welch(filtedbeta,nperseg=256)
    feature.append(max(psdtheta))
    feature.append(max(psdalpha))
    feature.append(max(psdbeta))
    return feature

In [5]:
if __name__ == '__main__':
    total=0
    path=r"C:\Users\chari\Downloads\project\EEG-ECG-Emotions\Data\DREAMER.mat"
    data=sio.loadmat(path)
    print("EEG signals are being feature extracted...")
    EEG_tmp=np.zeros((23,18,42))
    for k in range(0,23):
        for j in range(0,18):
            for i in range(0,14):
                B,S=[],[]
                basl=data['DREAMER'][0,0]['Data'][0,k]['EEG'][0,0]['baseline'][0,0][j,0][:,i]
                stim=data['DREAMER'][0,0]['Data'][0,k]['EEG'][0,0]['stimuli'][0,0][j,0][:,i]
                B=preprocessing(basl,B)
                S=preprocessing(stim,S)
                Extrod=np.divide(S,B)
                total+=1
                EEG_tmp[k,j,3*i]=Extrod[0]
                EEG_tmp[k,j,3*i+1]=Extrod[1]
                EEG_tmp[k,j,3*i+2]=Extrod[2]
                print("\rprogress: %d%%" %(total/(23*18*14)*100),end="")
    col=[]
    for i in range(0,14):
        col.append('psdtheta_'+str(i + 1)+'_un')
        col.append('psdalpha_'+str(i + 1)+'_un')
        col.append('psdbeta_'+str(i + 1)+'_un')
    EEG=pd.DataFrame(EEG_tmp.reshape((23 * 18,EEG_tmp.shape[2])),columns=col)
    scaler=pre.StandardScaler()
    for i in range(len(col)):
        EEG[col[i][:-3]]=scaler.fit_transform(EEG[[col[i]]])
    EEG.drop(col,axis=1,inplace=True)
    print(EEG)
    EEG.to_csv('EEG.csv')

EEG signals are being feature extracted...
progress: 100%     psdtheta_1  psdalpha_1  psdbeta_1  psdtheta_2  psdalpha_2  psdbeta_2  \
0     -0.137607   -0.137238  -0.116997   -0.080612   -0.081118  -0.108773   
1     -0.139174   -0.138685  -0.117599   -0.096207   -0.096338  -0.110293   
2     -0.127690   -0.127539  -0.110420   -0.101203   -0.101259  -0.098381   
3     -0.158972   -0.158123  -0.131111   -0.100674   -0.100739  -0.097964   
4     -0.110480   -0.110713  -0.099031   -0.111506   -0.111337  -0.089051   
..          ...         ...        ...         ...         ...        ...   
409   -0.176673   -0.175428  -0.142822   -0.119956   -0.118711  -0.111964   
410   -0.143317   -0.142784  -0.120602   -0.111610   -0.111338  -0.079139   
411   -0.153161   -0.152451  -0.127275   -0.055160   -0.047706  -0.035399   
412   -0.221893   -0.219656  -0.172804   -0.092856   -0.088311  -0.070856   
413   -0.128817   -0.128628  -0.111106   -0.141103   -0.139356  -0.113828   

     psdtheta_3  p